In [11]:
import pennylane as qml
from pennylane import numpy as np

symbols = ["Li", "H"]
geometry = np.array([[0.0, 0.0, 0.0],
                     [1.5478, 0.0,  0.0]])

H, qubits = qml.qchem.molecular_hamiltonian( symbols,geometry, charge=0)#
print(H)

  (-3.810362613555178) [I0]
+ (-0.38436821691657874) [Z10]
+ (-0.38436821691657874) [Z11]
+ (-0.21264344811286978) [Z9]
+ (-0.21264344811286975) [Z8]
+ (-0.21264344811283498) [Z6]
+ (-0.2126434481128348) [Z7]
+ (-0.1816506891674392) [Z4]
+ (-0.1816506891674392) [Z5]
+ (-0.0883812776805312) [Z2]
+ (-0.08838127768053108) [Z3]
+ (1.0705620972860068) [Z0]
+ (1.0705620972860068) [Z1]
+ (-0.00858536713717779) [Y2 Y4]
+ (-0.00858536713717779) [X2 X4]
+ (-0.004014999673108501) [Y1 Y3]
+ (-0.004014999673108501) [X1 X3]
+ (-0.0013293372406911821) [Y3 Y5]
+ (-0.0013293372406911821) [X3 X5]
+ (0.046126191918659215) [Y0 Y2]
+ (0.046126191918659215) [X0 X2]
+ (0.05551635658238005) [Z4 Z10]
+ (0.05551635658238005) [Z5 Z11]
+ (0.0598863933203812) [Z4 Z6]
+ (0.0598863933203812) [Z5 Z7]
+ (0.05988639332039454) [Z4 Z8]
+ (0.05988639332039454) [Z5 Z9]
+ (0.06136971478306755) [Z2 Z4]
+ (0.06136971478306755) [Z3 Z5]
+ (0.062320744059522634) [Z4 Z11]
+ (0.062320744059522634) [Z5 Z10]
+ (0.06368212426504019) 

In [12]:
generators = qml.symmetry_generators(H)
paulixops = qml.paulix_ops(generators, qubits)

for idx, generator in enumerate(generators):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulixops[idx]}")

generator 1:   (1.0) [Z6 Z7], paulix_op: PauliX(wires=[7])
generator 2:   (1.0) [Z8 Z9], paulix_op: PauliX(wires=[9])
generator 3:   (1.0) [Z0 Z2 Z4 Z6 Z8 Z10], paulix_op: PauliX(wires=[10])
generator 4:   (1.0) [Z1 Z3 Z5 Z6 Z8 Z11], paulix_op: PauliX(wires=[11])


In [13]:
n_electrons = 4
paulix_sector = qml.qchem.optimal_sector(H, generators, n_electrons)
print(paulix_sector)

[1, 1, 1, 1]


In [14]:
H_tapered = qml.taper(H, generators, paulixops, paulix_sector)
print(H_tapered)

  ((-3.653889886512547+0j)) [I0]
+ ((-0.42528689622573906+0j)) [Z8]
+ ((-0.4252868962256695+0j)) [Z6]
+ ((-0.18165068916743904+0j)) [Z4]
+ ((-0.18165068916743904+0j)) [Z5]
+ ((-0.08838127768053113+0j)) [Z2]
+ ((-0.08838127768053104+0j)) [Z3]
+ ((-0.009004386488219438+0j)) [X4]
+ ((-0.0037480169719367735+0j)) [X0]
+ ((0.0032031872614365496+0j)) [X6]
+ ((0.0032031872614372596+0j)) [X8]
+ ((0.009700753255951986+0j)) [X5]
+ ((0.03455184452382081+0j)) [X1]
+ ((1.070562097286006+0j)) [Z0]
+ ((1.070562097286006+0j)) [Z1]
+ ((-0.030742839311595813+0j)) [Y2 Y3]
+ ((-0.008585367137177781+0j)) [Y2 Y4]
+ ((-0.008585367137177781+0j)) [X2 X4]
+ ((-0.00699592799179631+0j)) [X3 X4]
+ ((-0.006804387477142576+0j)) [Y4 Y5]
+ ((-0.006346043485170829+0j)) [Y0 Y1]
+ ((-0.004014999673108498+0j)) [Y1 Y3]
+ ((-0.004014999673108498+0j)) [X1 X3]
+ ((-0.0026479822363783415+0j)) [X1 X2]
+ ((-0.002015896474434107+0j)) [Y0 Y5]
+ ((-0.0013293372406911817+0j)) [Y3 Y5]
+ ((-0.0013293372406911817+0j)) [X3 X5]
+ ((0.0004

In [15]:
H_sparse = qml.SparseHamiltonian(H.sparse_matrix(), wires=H.wires)
H_tapered_sparse = qml.SparseHamiltonian(H_tapered.sparse_matrix(), wires=H_tapered.wires)

In [16]:
print("Eigenvalues of H:\n", qml.eigvals(H_sparse, k=16))
print("\nEigenvalues of H_tapered:\n", qml.eigvals(H_tapered_sparse, k=16))

Eigenvalues of H:
 [-7.65384245 -7.59078211 -7.59078211 -7.54661645 -7.53166293 -7.54661645
 -7.54661645 -7.5153166  -7.5153166  -7.5153166  -7.5153166  -7.50637246
 -7.50637246 -7.50637246 -7.50637246 -7.50637246]

Eigenvalues of H_tapered:
 [-7.65384245 -7.54661645 -7.53166293 -7.18978241 -7.14315406 -7.08036664
 -6.99262339 -6.93948971 -6.8732842  -6.76592959 -6.99523772 -6.96602135
 -6.99523772 -6.96602135 -6.84484249 -6.84484249]


In [17]:
state_tapered = qml.qchem.taper_hf(generators, paulixops, paulix_sector,
                                   num_electrons=n_electrons, num_wires=len(H.wires))
print(state_tapered)

[1 1 1 1 0 0 0 0]


In [18]:
singles, doubles = qml.qchem.excitations(n_electrons, len(H.wires))
tapered_doubles = [
    qml.taper_operation(qml.DoubleExcitation, generators, paulixops, paulix_sector,
                        wire_order=H.wires, op_wires=double) for double in doubles
]
tapered_singles = [
    qml.taper_operation(qml.SingleExcitation, generators, paulixops, paulix_sector,
                        wire_order=H.wires, op_wires=single) for single in singles
]

dev = qml.device("default.qubit", wires=H_tapered.wires)
@qml.qnode(dev, interface="autograd")
def tapered_circuit(params):
    qml.BasisState(state_tapered, wires=H_tapered.wires)
    for idx, tapered_op in enumerate(tapered_doubles + tapered_singles):
        tapered_op(params[idx])
    return qml.expval(H_tapered)



In [20]:
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)
params = np.zeros(len(doubles) + len(singles), requires_grad=True)

for n in range(1, 31):
    params, energy = optimizer.step_and_cost(tapered_circuit, params)
    if not n % 5:
        print(f"n: {n}, E: {energy:.8f} Ha, Params: {params}")

n: 5, E: -7.65188127 Ha, Params: [ 6.29029966e-03  0.00000000e+00  0.00000000e+00  4.48859892e-03
  0.00000000e+00  0.00000000e+00 -4.48553579e-03  3.61159093e-03
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3.60976464e-03  0.00000000e+00  0.00000000e+00  8.00564900e-03
  0.00000000e+00  0.00000000e+00  7.47051041e-03  0.00000000e+00
  0.00000000e+00  0.00000000e+00  9.74595617e-04  0.00000000e+00
  0.00000000e+00 -1.28159343e-02  0.00000000e+00  0.00000000e+00
  5.34716955e-03  6.32490814e-03  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  6.32498918e-03  0.00000000e+00
  0.00000000e+00  7.32563019e-03 -9.83250007e-04  0.00000000e+00
  0.00000000e+00  5.34449809e-03  0.00000000e+00  0.00000000e+00
 -1.28156282e-02 -6.32406809e-03  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -6.32421023e-03  0.00000000e+00
  0.00000000e+00 -7.32755683e-03  0.00000000e+00  0.00000000e+00
  7.46971972e-03  0.00000000e+00  0.00000000e+00  0.00000